In [ ]:
# Mount Google Drive to save model and checkpoints
from google.colab import drive
import os

print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Create output directory in Drive
drive_output_dir = "/content/drive/MyDrive/MedLlama3_FineTuned"
os.makedirs(drive_output_dir, exist_ok=True)

print(f" Google Drive mounted successfully!")
print(f" Model checkpoints will be saved to: {drive_output_dir}")

# Check Drive space
import shutil
total, used, free = shutil.disk_usage("/content/drive/MyDrive/")
print(f"\nGoogle Drive Storage:")
print(f"  Total: {total // (2**30)} GB")
print(f"  Used: {used // (2**30)} GB")
print(f"  Free: {free // (2**30)} GB")

if free < 10 * (2**30):  # Less than 10GB
    print(" WARNING: Less than 10GB free space. Training may fail!")
    print("   Please free up space in your Drive before proceeding.")
elif free < 20 * (2**30):  # Less than 20GB
    print("⚠️ Note: Training checkpoints can be large. Monitor space during training.")
else:
    print(" Sufficient space available for training.")

# Dataset path - Upload your dataset to Drive first!
dataset_path = "/content/drive/MyDrive/bio_mistral_qa_combined.csv"
if os.path.exists(dataset_path):
    print(f"\n✅ Dataset found at: {dataset_path}")
else:
    print(f"\n❌ Dataset NOT found at: {dataset_path}")
    print("📝 Please upload 'bio_mistral_qa_combined.csv' to your Drive root folder.")
    print("   Or update the dataset_path variable above to match your file location.")

🔗 Mounting Google Drive...
Mounted at /content/drive
✅ Google Drive mounted successfully!
📁 Model checkpoints will be saved to: /content/drive/MyDrive/MedLlama3_FineTuned

💾 Google Drive Storage:
  Total: 235 GB
  Used: 49 GB
  Free: 186 GB
✅ Sufficient space available for training.

✅ Dataset found at: /content/drive/MyDrive/bio_mistral_qa_combined.csv


In [ ]:
pip install datasets peft trl transformers pandas torch spacy nltk rouge_score bert_score sentence_transformers bitsandbytes accelerate


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 45.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9c0b0195f04ed044e2300a0df69becbe39ead51392eb9f018d5ba76e8f75cbc0
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import os
import json
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, PeftModelForCausalLM
from trl import SFTTrainer

from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
import json
import os

# Note: You may need to authenticate with Hugging Face to access Meta-Llama-3-8B
# Run: huggingface-cli login
# Or set HF_TOKEN environment variable

# Custom callback to monitor loss and stop training
class EarlyStoppingOnLossCallback(TrainerCallback):
    """
    Custom callback to save checkpoint and optionally stop training when loss reaches target range
    """
    def __init__(self, target_loss_min=0.1, target_loss_max=0.2, patience=100, auto_stop=False):
        self.target_loss_min = target_loss_min
        self.target_loss_max = target_loss_max
        self.patience = patience
        self.auto_stop = auto_stop
        self.steps_in_range = 0
        self.best_checkpoint_saved = False

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            current_loss = logs["loss"]
            current_step = state.global_step

            print(f"\n[Step {current_step}] Current Loss: {current_loss:.4f}")

            # Check if loss is in optimal range
            if self.target_loss_min <= current_loss <= self.target_loss_max:
                self.steps_in_range += 1
                print(f"✅ Loss in optimal range [{self.target_loss_min}-{self.target_loss_max}]! ({self.steps_in_range}/{self.patience} steps)")

                # Save checkpoint when first entering optimal range
                if not self.best_checkpoint_saved:
                    print(f"💾 Saving checkpoint at loss {current_loss:.4f}")
                    control.should_save = True
                    self.best_checkpoint_saved = True

                # Stop training if stayed in range for patience steps
                if self.auto_stop and self.steps_in_range >= self.patience:
                    print(f"\n🛑 Stopping training! Loss has been in optimal range for {self.patience} steps.")
                    print(f"Final loss: {current_loss:.4f}")
                    control.should_training_stop = True
            else:
                self.steps_in_range = 0

            # Warning for very low loss
            if current_loss < 0.05:
                print(f"⚠️ WARNING: Loss is very low ({current_loss:.4f})! Possible overfitting!")

        return control

In [ ]:
def create_prompt(instruction, input_text):
    """Format the instruction and input into a prompt"""
    if input_text:
        return f"{instruction}\n\n{input_text}"
    return instruction

def load_and_format_dataset(file_path, train_split=0.8, output_dir="data", max_samples=None):
    """Improved dataset preparation with optional sample limit"""
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(file_path)

    # Validate and filter
    required_columns = ["instruction", "input", "output"]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Dataset must contain {required_columns} columns")
    df = df[df["input"] != "No structured clinical data available."]

    # Optional: limit dataset size for faster training/testing
    if max_samples and len(df) > max_samples:
        df = df.sample(n=max_samples, random_state=42)
        print(f"Using {max_samples} samples for faster training")

    formatted_data = []
    for _, row in df.iterrows():
        # Create chat format
        user_msg = create_prompt(row["instruction"], row["input"])
        assistant_msg = row["output"]

        # Create both formats
        formatted_data.append({
            "messages": [
                {"role": "user", "content": user_msg},
                {"role": "assistant", "content": assistant_msg}
            ],
            "text": f"### User: {user_msg} ###\n### Assistant: {assistant_msg} ###"
        })

    # Split and save
    train_size = int(len(formatted_data) * train_split)
    for split, data in [("train", formatted_data[:train_size]),
                       ("validation", formatted_data[train_size:])]:
        with open(os.path.join(output_dir, f"{split}.jsonl"), "w") as f:
            for item in data:
                json.dump(item, f)
                f.write("\n")

    print(f"Saved {train_size} training and {len(formatted_data)-train_size} validation examples")
    return load_dataset("json", data_files={
        "train": os.path.join(output_dir, "train.jsonl"),
        "validation": os.path.join(output_dir, "validation.jsonl")
    })

In [ ]:
def preprocess_and_save_dataset(dataset, tokenizer, output_dir="preprocessed_data"):
    """Pre-tokenize and cache dataset"""
    os.makedirs(output_dir, exist_ok=True)
    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=512)  # Truncate to 512 tokens
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["messages", "text"])
    tokenized_dataset.save_to_disk(output_dir)
    return tokenized_dataset

def configure_qlora_model(model_name="meta-llama/Meta-Llama-3-8B"):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        dtype=torch.float16,
        trust_remote_code=True
    )
    model.config.pad_token_id = tokenizer.eos_token_id
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=4,  # Reduced from 8
        lora_alpha=8,  # Reduced from 16
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        bias="none",
        task_type="CAUSAL_LM",
        inference_mode=False,
        fan_in_fan_out=False,
        modules_to_save=["embed_tokens", "lm_head"]
    )

    print("Applying PEFT adapters to the model...")
    peft_model = get_peft_model(model, lora_config)
    print(f"[DEBUG] Type after get_peft_model: {type(peft_model)}")

    if not isinstance(peft_model, (PeftModel, PeftModelForCausalLM)):
        raise ValueError("Model is not a PEFT model instance!")
    else:
        print("[OK] Model wrapped with PEFT successfully.")

    print(peft_model.print_trainable_parameters())

    for name, param in peft_model.named_parameters():
        if 'lora' in name:
            param.requires_grad = True

    return peft_model, tokenizer

In [ ]:
import os
from huggingface_hub import login, HfApi, whoami

# 1) put your NEW token here (don’t share it)
os.environ["HUGGINGFACE_HUB_TOKEN"] = "HF_TOKEN_FROM_ENV"

# 2) login so the credential is cached
login(os.environ["HUGGINGFACE_HUB_TOKEN"])

# 3) sanity checks
print("whoami:", whoami())
api = HfApi()
# This will raise 401 (not authed) or 403 (no access) if something's off
print(api.model_info("meta-llama/Meta-Llama-3-8B", token=os.environ["HUGGINGFACE_HUB_TOKEN"]).sha)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


whoami: {'type': 'user', 'id': '67a65f47497698c82022ada1', 'name': 'AparnaSuresh', 'fullname': 'AparnaSuresh', 'isPro': False, 'avatarUrl': '/avatars/6fcc5210516443950f310cdd623057d8.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Project', 'role': 'fineGrained', 'createdAt': '2025-10-27T23:01:22.527Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '661f97d48e7f3438386f755d', 'type': 'model', 'name': 'meta-llama/Meta-Llama-3-8B'}, 'permissions': ['repo.content.read', 'discussion.write']}, {'entity': {'_id': '67a65f47497698c82022ada1', 'type': 'user', 'name': 'AparnaSuresh'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.serverless.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write', 'job.write']}]}}}}
8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920


In [ ]:
def setup_trainer(model, dataset, output_dir="/content/drive/MyDrive/MedLlama3_FineTuned"):
    if not isinstance(model, (PeftModel, PeftModelForCausalLM)):
        raise ValueError("Model is not a PEFT-wrapped instance! Cannot continue with training.")

    print(f"Model is a PEFT model: {isinstance(model, (PeftModel, PeftModelForCausalLM))}")

    # Detect if A100 GPU is available and optimize settings accordingly
    device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
    is_a100 = "A100" in device_name

    if is_a100:
        print("🚀 A100 GPU detected! Using optimized settings for faster training.")
        batch_size = 8  # A100 can handle larger batches
        gradient_accum = 1
        workers = 4
    else:
        print(f"📊 Using {device_name}. Standard settings applied.")
        batch_size = 8
        gradient_accum = 1
        workers = 4

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,  # Auto-adjusted for GPU
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accum,
        num_train_epochs=2,  # Keep at 1 epoch
        learning_rate=1e-4,  # Reduced from 3e-4 to prevent overfitting
        bf16=True,
        bf16_full_eval=True,
        save_strategy="steps",  # Changed to steps to save multiple checkpoints
        save_steps=1000,  # Save every 1000 steps for safety
        eval_strategy="no",  # Disable eval during training for speed
        load_best_model_at_end=False,
        logging_steps=100,  # Less frequent logging
        save_total_limit=3,  # Keep last 3 checkpoints (IMPORTANT!)
        push_to_hub=False,
        gradient_checkpointing=False,  # Disabled for speed
        optim="adamw_torch_fused",
        max_grad_norm=0.3,
        warmup_steps=100,  # Increased warmup for stability
        lr_scheduler_type="cosine",  # Cosine decay to prevent overfitting
        dataloader_num_workers=workers,  # More workers for A100
        dataloader_pin_memory=True,
        dataloader_prefetch_factor=2,  # Prefetch batches
        group_by_length=True,  # Group similar lengths for efficiency
        max_steps=-1,  # ADDED: Stop at 3000 steps to prevent overfitting
    )

    print(f"✅ Training config: Batch size={batch_size}, Gradient accum={gradient_accum}, Workers={workers}")

    def formatting_func(example):
        return "\n".join([
            f"### {msg['role'].capitalize()}: {msg['content']} ###"
            for msg in example["messages"]
        ])

    # Create callback to monitor loss and stop if in optimal range
    early_stop_callback = EarlyStoppingOnLossCallback(
        target_loss_min=0.1,      # Minimum acceptable loss
        target_loss_max=0.2,      # Maximum acceptable loss
        patience=50,              # Wait 50 steps in range before stopping
        auto_stop=True            # Set to True to auto-stop, False to just save checkpoint
    )

    print("Creating SFTTrainer with EarlyStoppingOnLoss callback...")
    print(f"📊 Will monitor loss and save checkpoint when in range [0.1-0.2]")
    print(f"🛑 Auto-stop enabled: Training will stop after 50 steps in optimal range")

    return SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"] if training_args.eval_strategy != "no" else None,
        args=training_args,
        formatting_func=formatting_func,
        peft_config=None,
        callbacks=[early_stop_callback],
    )

def main():
    torch.cuda.empty_cache()  # Clear GPU memory
    print("Loading and preparing dataset...")
    dataset = load_and_format_dataset("bio_mistral_qa_combined.csv", max_samples=None)  # Use full dataset
    print(f"Dataset loaded: {dataset}")

    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("Preprocessing and tokenizing dataset...")
    dataset = preprocess_and_save_dataset(dataset, tokenizer)
    print(f"Preprocessed dataset: {dataset}")

    # Clear tokenizer from memory before loading full model
    del tokenizer
    torch.cuda.empty_cache()

    print("Configuring QLoRA model...")
    model, tokenizer = configure_qlora_model()  # Load model only once
    print("QLoRA model configured successfully!")

    if not isinstance(model, (PeftModel, PeftModelForCausalLM)):
        raise ValueError("Model is not properly wrapped as a PEFT model!")

    print("Setting up trainer...")
    trainer = setup_trainer(model, dataset)
    print("Trainer configured successfully!")

    print("Starting training...")
    trainer.train()

    trainer.save_model()
    print(f"Model trained and saved to {trainer.args.output_dir}")

    return model, tokenizer, trainer

# Run training
model, tokenizer, trainer = main()

Loading and preparing dataset...
Saved 89530 training and 22383 validation examples


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 89530
    })
    validation: Dataset({
        features: ['messages', 'text'],
        num_rows: 22383
    })
})
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

...


[Step 20100] Current Loss: 0.6638

[Step 20200] Current Loss: 0.6594

[Step 20300] Current Loss: 0.6805

[Step 20400] Current Loss: 0.6382

[Step 20500] Current Loss: 0.7121

[Step 20600] Current Loss: 0.6814

[Step 20700] Current Loss: 0.6794

[Step 20800] Current Loss: 0.7158

[Step 20900] Current Loss: 0.7027

[Step 21000] Current Loss: 0.6845


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



[Step 21100] Current Loss: 0.6803

[Step 21200] Current Loss: 0.6639

[Step 21300] Current Loss: 0.6561

[Step 21400] Current Loss: 0.6985

[Step 21500] Current Loss: 0.6444

[Step 21600] Current Loss: 0.6947

[Step 21700] Current Loss: 0.6990

[Step 21800] Current Loss: 0.6955

[Step 21900] Current Loss: 0.6788

[Step 22000] Current Loss: 0.6747


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



[Step 22100] Current Loss: 0.6922

[Step 22200] Current Loss: 0.7135

[Step 22300] Current Loss: 0.6658
Model trained and saved to /content/drive/MyDrive/MedLlama3_FineTuned
